# Assignment 3

## Instructions
- Run this notebook on ```Google Colab(preferable)```
- Write your code and analysis in the indicated cells.
- Ensure that this notebook runs without errors when the cells are run in sequence.
- Do not attempt to change the contents of other cells. 

## Packages Used
- sklearn [link](https://scikit-learn.org/)
- Keras [link](https://keras.io/guides/)

## Submission
- Rename the notebook to `<roll_number>_Assignment3_Q3.ipynb`.


## Question 3
Fake news is a widespread problem and there are many methods for combating it.
You have to build a fake news detection system using a ML model. Train any ML model (ANN, LSTM) over the given Dataset.
The dataset has short statements spoken by people and has the meta-information and corresponding label for those sentences. 
Your target is label column which has 6 labels(in the increasing order of truthfullness): pants-fire, false, barely-true, half-true, mostly-true, true.

The features are 'statement', 'subject', 'speaker', 'job', 'state', 'party', 'barely_true_c', 'false_c', 'half_true_c', 'mostly_true_c', 'pants_on_fire_c', 'venue' and the target is column "label".

The statement is made by speaker whose job, party are given along with 6 columns which are an account of the  type of news(labels) the person has shared before. 
The person who has shared fake content before is likely to share it in future and this can be accounted by the ML model as a feature. Column barely_true_c contains how many barely_true news has the speaker shared (and so is with column X_c, value of X_c is number of X the person shared).


You have to perform two tasks:
* task1: Binary classification <br>
Classify the given news as true/false. Take the labels pants-fire, false, barely-true as false and rest (half-true, mostly-true, true) as true.
* task2: Six-way classification <br>
Classify the given news into six-classes "pants-fire, false, barely-true, half-true, mostly-true, true".

For each of the tasks:
1) Experiment with depth of network and try to fine-tune hyperparameters reporting your observations. <br>
2) Report the accuracy, f1-score, confusion matrix on train, val and test sets. <br>
3) Experiment with bag-of-words, glove and bert embeddings(code given in the below notebook) and report results. <br> Comment on what is the affect of embedding on the results.

The pre-processing code is provided, you need to write the training and test.

Note: You are supposed to train on trainset, fine-tune on val and just eval on test set. If found that you trained on val/test sets, the penalty will be incurred.

In [1]:
# !pip install numpy
# !pip install tensorflow
# !pip install re
# !pip install nltk
# !pip install keras
# !pip install sklearn

In [2]:
# Importing libraries
import numpy as np
import pandas as pd
from tensorflow import keras  #feel free to use any other library
import numpy as np

import re
import nltk
import numpy as np
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
nltk.download('stopwords')
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Activation
from sklearn.metrics import multilabel_confusion_matrix,accuracy_score,f1_score


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
train = pd.read_csv('train.csv')
val = pd.read_csv('val.csv')
test = pd.read_csv('test.csv')

In [4]:
# Dropping the 'id' column
train.drop('id', axis = 1, inplace = True)
test.drop('id', axis = 1, inplace = True)
val.drop('id', axis = 1, inplace = True)

In [5]:
train.head(5)

,label,statement,subject,speaker,job,state,party,barely_true_c,false_c,half_true_c,mostly_true_c,pants_on_fire_c,venue
0,False,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0,1,0,0,0,a mailer
1,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0,0,1,1,0,a floor speech.
2,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70,71,160,163,9,Denver
3,False,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7,19,3,5,44,a news release
4,half-true,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15,9,20,19,2,an interview on CNN


In [6]:
# Checking the shape of data
print(train.shape)
print(val.shape)
print(test.shape)

(10269, 13)
(1284, 13)
(1283, 13)


## Clean and pre-process data
* Replace missing values
* Remove numbers and special characters
* Convert to upper-case

We experiment with two types of processing, one directly appending the other attributes like subject, job, state, party to sentence and then applying bag of words on it.

Other being encoding sentence with glove embeddings and passing just that.

In [7]:

def dataPreprocessing(data):
    '''Function for cleaning the dataset
    '''
    corpus = []
    # Missing values
    data["job"].fillna("no-job", inplace = True)
    data["state"].fillna("no-state", inplace = True)

    for x in range(data.shape[0]):
        statement = re.sub('[^a-zA-Z]', ' ', data['statement'][x]) # Removing all numbers and special characters
        statement = statement.lower() # Converting uppercase to lowercase
        statement = statement.split()
        
        # you can experiment with any other stemmers
        ps = PorterStemmer()
        statement = [ps.stem(word) for word in statement if not word in set(stopwords.words('english'))] # Stemming the dataset and removing stopwords
        statement = ' '.join(statement)
        subject = data['subject'][x].replace(',', ' ')
        speaker = data['speaker'][x]
        job = data['job'][x].lower()
        # job = job.replace(' ', '-')
        state = data['state'][x].lower()
        party = data['party'][x].lower()
        corpus.append(statement + ' '  + subject + ' ' + job + ' ' + state + ' ' + party)
    return corpus

In [8]:
x_train = dataPreprocessing(train)
x_val = dataPreprocessing(val) 
x_test = dataPreprocessing(test) 

In [9]:
len(x_train), len(x_val), len(x_test)

(10269, 1284, 1283)

In [10]:
corpus = x_train + x_val + x_test

## Using bag-of-words embedding


In [11]:
# Converting the corpus into bag-of-words
cv = CountVectorizer(max_features = 8000)
X = cv.fit_transform(corpus).toarray()

In [12]:
X

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [13]:
X.shape

(12836, 8000)

In [14]:
train.columns

Index(['label', 'statement', 'subject', 'speaker', 'job', 'state', 'party',
       'barely_true_c', 'false_c', 'half_true_c', 'mostly_true_c',
       'pants_on_fire_c', 'venue'],
      dtype='object')

In [15]:
# Selecting the columns 'barely_true_c',	'false_c',	'half_true_c',	'mostly_true_c',	'pants_on_fire_c'
label_cols = ['barely_true_c', 'false_c', 'half_true_c', 'mostly_true_c',
       'pants_on_fire_c']
x_train2 = train[label_cols]
x_val2 = val[label_cols]
x_test2 = test[label_cols]

In [16]:
# Stacking x_train and x_train2 horizontally
x_train_bow = np.hstack((X[:len(x_train)], x_train2))
x_val_bow = np.hstack((X[len(x_train):len(x_train)+len(x_val)], x_val2))
x_test_bow = np.hstack((X[len(x_train)+len(x_val):], x_test2))

In [17]:
x_train_bow.shape

(10269, 8005)

## Use of Glove Embedding


download glove embeddings from 'https://nlp.stanford.edu/data/glove.6B.zip','glove.6B.zip'
and place in your current working folder


In [18]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

--2022-04-16 18:15:35--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2022-04-16 18:15:35--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2022-04-16 18:15:35--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip.1’

glove

In [19]:
!unzip "glove.6B.zip" -d "glove"

Archive:  glove.6B.zip
replace glove/glove.6B.50d.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [21]:
emmbed_dict = {}
with open('glove/glove.6B.200d.txt','r') as f:
  for line in f:
    values = line.split()
    word = values[0]
    vector = np.asarray(values[1:],'float32')
    emmbed_dict[word]=vector


In [22]:
emmbed_dict['oov'] = np.zeros(200)

In [23]:
!pip install sentence_transformers

In [24]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

In [25]:
from nltk.tokenize import word_tokenize
nltk.download('punkt')
def dataPreprocessing_glove(data):
    corpus = []
    # Missing values
    data["job"].fillna("no-job", inplace = True)
    data["state"].fillna("no-state", inplace = True)

    for x in range(data.shape[0]):
        statement = re.sub('[^a-zA-Z]', ' ', data['statement'][x]) # Removing all numbers and special characters
        statement = statement.lower() # Converting uppercase to lowercase
        statement = word_tokenize(statement)

        embed_statement = []
        for w in statement:
            if w in emmbed_dict:
                embed_statement.append(emmbed_dict[w])
            else:
                embed_statement.append(emmbed_dict['oov'])
         
        # bonus: Think how you can encode the below features(hint: look upon label encoding or training your own word2vec or any other embedding model)
    
#         subject = data['subject'][x].replace(',', ' ')
#         speaker = data['speaker'][x]
#         job = data['job'][x].lower()
#         # job = job.replace(' ', '-')
#         state = data['state'][x].lower()
#         party = data['party'][x].lower()
        corpus.append(embed_statement)
    
    return np.array(corpus)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [26]:
x_train_glove = dataPreprocessing_glove(train)
x_val_glove = dataPreprocessing_glove(val) 
x_test_glove = dataPreprocessing_glove(test) 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [27]:
x_train_glove = np.hstack((x_train_glove.reshape(-1,1), x_train2))
x_val_glove = np.hstack((x_val_glove.reshape(-1,1), x_val2))
x_test_glove = np.hstack((x_test_glove.reshape(-1,1), x_test2))

## Use of bert embeddings
note: we used our pre-processed code for bow which has the attributed appended to end the end of sentence. 

In [28]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

x_train_bert = np.hstack((model.encode(x_train), x_train2))
x_val_bert = np.hstack((model.encode(x_val), x_val2))
x_test_bert = np.hstack((model.encode(x_test), x_test2))

Now use the above 3 types of embedded inputs(bow, glove, bert embeddings) for the 2 classification tasks and compare their outputs


# Six-way classification

## Preprocessing

In [29]:
num_classes = 6
# Preprocessing function for the labels
def categorize(data):
    y = data["label"].tolist()

    # Encoding the Dependent Variable
    labelencoder_y = LabelEncoder()
    y = labelencoder_y.fit_transform(y)

    # Converting to binary class matrix
    y = np_utils.to_categorical(y, num_classes)
    return y

In [30]:
y_train_six_way = categorize(train)
y_test_six_way = categorize(test)
y_val_six_way = categorize(val)

Build a model and pass bow, glove and bert embedded inputs: x_train_bow, x_train_glove, x_train_bert(similarly validate for val and report results on test)


## Model

In [31]:
x_train_bow.shape

(10269, 8005)

In [32]:
x_train_bert.shape

(10269, 389)

In [33]:
x_train_glove.shape

(10269, 6)

In [34]:
final_x_train_glove=np.zeros((10269,65,200))
for i in range(x_train_glove.shape[0]):
  num_array=np.array(x_train_glove[i][0])
  final_x_train_glove[i,:num_array.shape[0],:]=num_array
final_x_train_glove = final_x_train_glove.reshape(*final_x_train_glove.shape[:-2], -1)

In [35]:
final_x_train_glove.shape

(10269, 13000)

In [36]:
x_test_glove.shape

(1283, 6)

In [37]:
final_x_test_glove=np.zeros((1283,65,200))
for i in range(x_test_glove.shape[0]):
  num_array=np.array(x_test_glove[i][0])
  final_x_test_glove[i,:num_array.shape[0],:]=num_array
final_x_test_glove = final_x_test_glove.reshape(*final_x_test_glove.shape[:-2], -1)

In [38]:
final_x_test_glove.shape

(1283, 13000)

In [39]:
x_val_glove.shape

(1284, 6)

In [40]:
final_x_val_glove=np.zeros((1284,65,200))
for i in range(x_val_glove.shape[0]):
  num_array=np.array(x_val_glove[i][0])
  final_x_val_glove[i,:num_array.shape[0],:]=num_array
final_x_val_glove = final_x_val_glove.reshape(*final_x_val_glove.shape[:-2], -1)

In [41]:
final_x_val_glove.shape

(1284, 13000)

In [42]:
def print_results(ypred,yactual):
  ypred=np.argmax(ypred, axis=1)
  yactual=np.argmax(yactual, axis=1)
  mconfusion_mat = multilabel_confusion_matrix(yactual,ypred)
  print("Confusion Matrix:",mconfusion_mat)
  f1score = f1_score(yactual, ypred,average=None)
  print("F1_Score: ",f1score )

In [43]:
# BERT EMBEDDING
model = Sequential()
model.add(LSTM(100,input_shape=(389,1)))
model.add(Dense(6,activation='softmax'))
model.compile(loss='mse',optimizer ='adam',metrics=['accuracy'])
model.fit(x_train_bert,y_train_six_way,epochs=10,validation_data=(x_val_bert,y_val_six_way),verbose=1)
scores = model.evaluate(x_test_bert, y_test_six_way, batch_size=128)
# loss, accuracy, f1_score, precision, recall
print("Results For BERT")
print('Accuracy: {}'.format(scores[1]))
y_pred = model.predict(x_test_bert)
print_results(y_pred,y_test_six_way)

Epoch 1/10
321/321 [==============================] - 41s 99ms/step - loss: 0.1353 - accuracy: 0.2423 - val_loss: 0.1319 - val_accuracy: 0.3045
Epoch 2/10
321/321 [==============================] - 29s 91ms/step - loss: 0.1258 - accuracy: 0.3521 - val_loss: 0.1198 - val_accuracy: 0.4042
Epoch 3/10
321/321 [==============================] - 29s 90ms/step - loss: 0.1092 - accuracy: 0.4406 - val_loss: 0.1062 - val_accuracy: 0.4579
Epoch 4/10
321/321 [==============================] - 29s 89ms/step - loss: 0.1061 - accuracy: 0.4542 - val_loss: 0.1049 - val_accuracy: 0.4626
Epoch 5/10
321/321 [==============================] - 28s 88ms/step - loss: 0.1052 - accuracy: 0.4528 - val_loss: 0.1042 - val_accuracy: 0.4556
Epoch 6/10
321/321 [==============================] - 28s 88ms/step - loss: 0.1049 - accuracy: 0.4528 - val_loss: 0.1044 - val_accuracy: 0.4556
Epoch 7/10
321/321 [==============================] - 28s 87ms/step - loss: 0.1046 - accuracy: 0.4565 - val_loss: 0.1046 - val_accuracy:

In [44]:
# BOW EMBEDDING
model = Sequential()
model.add(Dense(5000,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(6,activation='softmax'))
model.compile(loss='mse',optimizer ='adam',metrics=['accuracy'])
model.fit(x_train_bow,y_train_six_way,epochs=10,validation_data=(x_val_bow,y_val_six_way),verbose=1)
scores = model.evaluate(x_test_bow, y_test_six_way, batch_size=128)
# loss, accuracy, f1_score, precision, recall
print("Results For BOW")
print('Accuracy : {}'.format(scores[1]))
y_pred = model.predict(x_test_bow)
print_results(y_pred,y_test_six_way)

Epoch 1/10
321/321 [==============================] - 7s 21ms/step - loss: 0.1570 - accuracy: 0.3473 - val_loss: 0.1442 - val_accuracy: 0.4167
Epoch 2/10
321/321 [==============================] - 7s 20ms/step - loss: 0.1258 - accuracy: 0.5086 - val_loss: 0.1536 - val_accuracy: 0.3988
Epoch 3/10
321/321 [==============================] - 7s 21ms/step - loss: 0.1132 - accuracy: 0.5819 - val_loss: 0.1534 - val_accuracy: 0.3793
Epoch 4/10
321/321 [==============================] - 7s 20ms/step - loss: 0.1065 - accuracy: 0.6270 - val_loss: 0.1665 - val_accuracy: 0.3917
Epoch 5/10
321/321 [==============================] - 7s 21ms/step - loss: 0.0992 - accuracy: 0.6644 - val_loss: 0.1860 - val_accuracy: 0.3637
Epoch 6/10
321/321 [==============================] - 7s 20ms/step - loss: 0.0978 - accuracy: 0.6784 - val_loss: 0.1860 - val_accuracy: 0.3715
Epoch 7/10
321/321 [==============================] - 7s 20ms/step - loss: 0.1021 - accuracy: 0.6689 - val_loss: 0.1868 - val_accuracy: 0.3692

In [47]:
#GLOVE EMBEDDING
model = Sequential()
model.add(Dense(5000,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(2500,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(6,activation='softmax'))
model.compile(loss='mse',optimizer ='adam',metrics=['accuracy'])
model.fit(final_x_train_glove,y_train_six_way,epochs=10,validation_data=(final_x_val_glove,y_val_six_way),verbose=1)
scores = model.evaluate(final_x_test_glove, y_test_six_way, batch_size=128)
# loss, accuracy, f1_score, precision, recall
print("Results For GLOVE")
print('Accuracy: {}'.format(scores[1]))
y_pred = model.predict(final_x_test_glove)
print_results(y_pred,y_test_six_way)


Epoch 1/10
321/321 [==============================] - 13s 37ms/step - loss: 0.2791 - accuracy: 0.1609 - val_loss: 0.2718 - val_accuracy: 0.1846
Epoch 2/10
321/321 [==============================] - 12s 36ms/step - loss: 0.2795 - accuracy: 0.1614 - val_loss: 0.2718 - val_accuracy: 0.1846
Epoch 3/10
321/321 [==============================] - 11s 36ms/step - loss: 0.2795 - accuracy: 0.1614 - val_loss: 0.2718 - val_accuracy: 0.1846
Epoch 4/10
321/321 [==============================] - 12s 37ms/step - loss: 0.2795 - accuracy: 0.1614 - val_loss: 0.2718 - val_accuracy: 0.1846
Epoch 5/10
321/321 [==============================] - 13s 39ms/step - loss: 0.2795 - accuracy: 0.1614 - val_loss: 0.2718 - val_accuracy: 0.1846
Epoch 6/10
321/321 [==============================] - 13s 41ms/step - loss: 0.2795 - accuracy: 0.1614 - val_loss: 0.2718 - val_accuracy: 0.1846
Epoch 7/10
321/321 [==============================] - 12s 36ms/step - loss: 0.2795 - accuracy: 0.1614 - val_loss: 0.2718 - val_accuracy:

# Binary Classification

## Preprocessing

In [48]:
num_classes = 2

In [49]:
# Function for preprocessing labels
def dataPreprocessingBinary(data):
    y = data["label"].tolist()

    # Changing the 'half-true', 'mostly-true', barely-true', 'pants-fire' labels to True/False for Binary Classification
    for x in range(len(y)):
        if(y[x] == 'half-true'):
            y[x] = 'True'
        elif(y[x] == 'mostly-true'):
            y[x] = 'True'
        elif(y[x] == 'barely-true'):
            y[x] = 'False'
        elif(y[x] == 'pants-fire'):
            y[x] = 'False'

    # Converting the lables into binary class matrix
    labelencoder_y = LabelEncoder()
    y = labelencoder_y.fit_transform(y)
    y = np_utils.to_categorical(y, num_classes)
    return y

In [50]:
y_train_binary = dataPreprocessingBinary(train)
y_test_binary = dataPreprocessingBinary(test)
y_val_binary = dataPreprocessingBinary(val)

## Model
Build a model and pass bow, glove and bert embedded inputs: x_train_bow, x_train_glove, x_train_bert(similarly validate for val and report results on test)


In [51]:
## write your code here
# Initialize hyperparameters
# Create model
# train
# test
# report accuracy, f1-score and confusion matrix

In [52]:
# BERT EMBEDDING
model = Sequential()
model.add(LSTM(100,input_shape=(389,1)))
model.add(Dense(2,activation='softmax'))
model.compile(loss='binary_crossentropy',optimizer ='adam',metrics=['accuracy'])
model.fit(x_train_bert,y_train_binary,epochs=10,validation_data=(x_val_bert,y_val_binary),verbose=1)
scores = model.evaluate(x_test_bert, y_test_binary, batch_size=128)
print("Results For BERT")
print('Accuracy: {}'.format(scores[1]))
y_pred = model.predict(x_test_bert)
print_results(y_pred,y_test_binary)

Epoch 1/10
321/321 [==============================] - 32s 92ms/step - loss: 0.6511 - accuracy: 0.6200 - val_loss: 0.6227 - val_accuracy: 0.6612
Epoch 2/10
321/321 [==============================] - 29s 91ms/step - loss: 0.5696 - accuracy: 0.7045 - val_loss: 0.5686 - val_accuracy: 0.6869
Epoch 3/10
321/321 [==============================] - 28s 88ms/step - loss: 0.5416 - accuracy: 0.7106 - val_loss: 0.5317 - val_accuracy: 0.7079
Epoch 4/10
321/321 [==============================] - 28s 88ms/step - loss: 0.5263 - accuracy: 0.7181 - val_loss: 0.5244 - val_accuracy: 0.7017
Epoch 5/10
321/321 [==============================] - 28s 88ms/step - loss: 0.5192 - accuracy: 0.7248 - val_loss: 0.5171 - val_accuracy: 0.7033
Epoch 6/10
321/321 [==============================] - 28s 88ms/step - loss: 0.5121 - accuracy: 0.7295 - val_loss: 0.5155 - val_accuracy: 0.7118
Epoch 7/10
321/321 [==============================] - 28s 87ms/step - loss: 0.5086 - accuracy: 0.7305 - val_loss: 0.5145 - val_accuracy:

In [53]:
# BOW EMBEDDING
model = Sequential()
model.add(Dense(5000,activation='relu'))
model.add(Dropout(0.7))
model.add(Dense(2000,activation='relu'))
model.add(Dropout(0.7))
model.add(Dense(2,activation='softmax'))
model.compile(loss='binary_crossentropy',optimizer ='adam',metrics=['accuracy'])
model.fit(x_train_bow,y_train_binary,epochs=10,validation_data=(x_val_bow,y_val_binary),verbose=1)
scores = model.evaluate(x_test_bow, y_test_binary, batch_size=128)
# loss, accuracy, f1_score, precision, recall
print("Results for BOW")
print('Accuracy for BOW EMBEDDING: {}'.format(scores[1]))
y_pred = model.predict(x_test_bow)
print_results(y_pred,y_test_binary)

Epoch 1/10
321/321 [==============================] - 9s 26ms/step - loss: 0.9249 - accuracy: 0.6684 - val_loss: 0.5889 - val_accuracy: 0.6721
Epoch 2/10
321/321 [==============================] - 8s 26ms/step - loss: 0.5280 - accuracy: 0.7299 - val_loss: 0.5665 - val_accuracy: 0.7126
Epoch 3/10
321/321 [==============================] - 8s 26ms/step - loss: 0.4510 - accuracy: 0.7726 - val_loss: 0.5913 - val_accuracy: 0.6893
Epoch 4/10
321/321 [==============================] - 8s 26ms/step - loss: 0.3980 - accuracy: 0.8033 - val_loss: 0.6941 - val_accuracy: 0.6970
Epoch 5/10
321/321 [==============================] - 8s 26ms/step - loss: 0.3426 - accuracy: 0.8319 - val_loss: 0.8621 - val_accuracy: 0.6838
Epoch 6/10
321/321 [==============================] - 8s 26ms/step - loss: 0.3030 - accuracy: 0.8462 - val_loss: 0.9661 - val_accuracy: 0.6822
Epoch 7/10
321/321 [==============================] - 8s 26ms/step - loss: 0.2752 - accuracy: 0.8654 - val_loss: 1.1151 - val_accuracy: 0.6659

In [54]:
#GLOVE EMBEDDING
model = Sequential()
model.add(Dense(5000,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(2500,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss='binary_crossentropy',optimizer ='adam',metrics=['accuracy'])
model.fit(final_x_train_glove,y_train_binary,epochs=10,validation_data=(final_x_val_glove,y_val_binary),verbose=1)
scores = model.evaluate(final_x_test_glove, y_test_binary, batch_size=128)
# loss, accuracy, f1_score, precision, recall
print("Results For GLOVE")
print('Accuracy: {}'.format(scores[1]))
y_pred = model.predict(final_x_test_glove)
print_results(y_pred,y_test_binary)

Epoch 1/10
321/321 [==============================] - 12s 36ms/step - loss: 0.7015 - accuracy: 0.5769 - val_loss: 0.6809 - val_accuracy: 0.5335
Epoch 2/10
321/321 [==============================] - 12s 36ms/step - loss: 0.6274 - accuracy: 0.6533 - val_loss: 0.6655 - val_accuracy: 0.5880
Epoch 3/10
321/321 [==============================] - 12s 36ms/step - loss: 0.5220 - accuracy: 0.7446 - val_loss: 0.7270 - val_accuracy: 0.5911
Epoch 4/10
321/321 [==============================] - 11s 36ms/step - loss: 0.3267 - accuracy: 0.8617 - val_loss: 0.9770 - val_accuracy: 0.5849
Epoch 5/10
321/321 [==============================] - 11s 36ms/step - loss: 0.1470 - accuracy: 0.9422 - val_loss: 1.3607 - val_accuracy: 0.5833
Epoch 6/10
321/321 [==============================] - 12s 36ms/step - loss: 0.0970 - accuracy: 0.9633 - val_loss: 1.6143 - val_accuracy: 0.5732
Epoch 7/10
321/321 [==============================] - 11s 35ms/step - loss: 0.0658 - accuracy: 0.9772 - val_loss: 1.7396 - val_accuracy: